In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm sorry, I cannot provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Munich today.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 17, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-81bca48a-7e12-4305-8bdc-48555318f85d-0', usage_metadata={'input_tokens': 17, 'output_tokens': 36, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TFiMxSATXDg1lk86S9qLOQLb', 'function': {'arguments': '{"location":"munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 84, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3da53266-d896-403a-8f1e-7a7ebb844d44-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': 'call_TFiMxSATXDg1lk86S9qLOQLb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 16, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_TFiMxSATXDg1lk86S9qLOQLb',
  'type': 'tool_call'}]

In [8]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6ekf9OKcx3eJBuz0jJm7IvZr', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_yW63tlzjYYYY5DElHm57e6qe', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cafc3dd1-8c96-4019-835e-b971be36f781-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': 'call_6ekf9OKcx3eJBuz0jJm7IvZr', 'type': 'tool_call'}, {'name': 

In [9]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_6ekf9OKcx3eJBuz0jJm7IvZr',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_yW63tlzjYYYY5DElHm57e6qe',
  'type': 'tool_call'}]

In [10]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [11]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_A0wS51Tg2NOZ0KLuZpIwZweg', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_pqVQ1xTLiESGeLTYH67jWK5f', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-49e9836e-65aa-483b-bde6-

In [12]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [13]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': 'call_A0wS51Tg2NOZ0KLuZpIwZweg',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'munich', 'seating_type': 'outdoor'},
  'id': 'call_pqVQ1xTLiESGeLTYH67jWK5f',
  'type': 'tool_call'}]

In [14]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [15]:
messages

[HumanMessage(content='How will the weather be in munich today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_A0wS51Tg2NOZ0KLuZpIwZweg', 'function': {'arguments': '{"location": "munich"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_pqVQ1xTLiESGeLTYH67jWK5f', 'function': {'arguments': '{"location": "munich", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-49e9836e-65aa-483b-bde6-

In [16]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is 15 degrees Celsius and cloudy. Additionally, there are still seats available outdoors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 177, 'total_tokens': 201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-27213463-872a-4f91-b2c5-1f0c5da8caeb-0', usage_metadata={'input_tokens': 177, 'output_tokens': 24, 'total_tokens': 201, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})